<a href="https://colab.research.google.com/github/daavo03/NLP/blob/main/Rule_based_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk
!pip install wikipedia

# Install packages

In [ ]:
import nltk
import string
import random
import re, unicodedata
import warnings
import wikipedia as wk

from collections import defaultdict
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

warnings.filterwarnings("ignore")

# Download datasets

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

# Input data to use

In [ ]:
# Open the data and makesure there's no capital letters in the data
data = open('./drive/MyDrive/colab_files/HR.txt', 'r', errors='ignore')
raw = data.read()
raw = raw.lower()

In [ ]:
raw[:100]

'human resource analytics is at the intersection of three bodies of knowledge:\n\nhuman resource manage'

## Tokenize the data

In [ ]:
sent_tokens = nltk.sent_tokenize(raw)

In [ ]:
def normalize(text):
    # Remove the puntuaction dictionary
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

    # Develop the word tokenization
    word_token = nltk.word_tokenize(text.lower().translate(remove_punct_dict))

    # Remove ascii
    new_words = []
    for word in word_token:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)

    # Remove tags
    rmv = []
    for w in new_words:
        text = re.sub("&lt;/?.*?&gt;", "&lt;&gt;", w)
        rmv.append(text)

    # Lemmatization
    tag_map = defaultdict(lambda:wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV

    lmtzr = WordNetLemmatizer()
    lemma_list = []
    rmv = [i for i in rmv if i]
    for token, tag in nltk.pos_tag(rmv):
        lemma = lmtzr.lemmatize(token, tag_map[tag[0]])
        lemma_list.append(lemma)

    return lemma_list



In [ ]:
welcome_input = ("hello", "hi", "greetings", "sup", "what's up", "hey")
welcome_response = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

def welcome(user_response):
    for word in user_response.split():
        if word.lower() in welcome_input:
            return random.choice(welcome_response)

In [ ]:
def generateResponse(user_response):
    robo_response = ''
    sent_tokens.append(user_response)

    TfidfVec = TfidfVectorizer(tokenizer=normalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)

    vals = linear_kernel(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]

    if (req_tfidf == 0) or "tell me about" in user_response:
        print("Checking Wikipedia")
        if user_response:
            robo_response = wikipedia_data(user_response)
            return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

def wikipedia_data(input):
    reg_ex = re.search('tell me about(.*)', input)

    try:
        if reg_ex:
            topic = reg_ex.group(1)
            wiki = wk.summary(topic, sentences=3)
            return wiki
    except Exception as e:
        print("No content available")

In [ ]:
flag = True
print("My name us chatty and I'm a chatbot")

while flag:
    user_response = input().lower()

    if user_response not in ['bye', 'shutdown', 'exit', 'quit']:
        if (user_response == 'thanks' or user_response=='thank you'):
            flag = False
            print("Chattybot : You'r welcome")
        else:
            if(welcome(user_response) != None):
                print("Chattybot : "+welcome(user_response))
            else:
                print("Chattybot : ", end="")
                print(generateResponse(user_response))
                sent_tokens.remove(user_response)
    else:
        flag = False
        print("Chattybot : Bye!")

My name us chatty and I'm a chatbot
daavo
Chattybot : Checking Wikipedia
None
hola
Chattybot : Checking Wikipedia
None
What is HR
Chattybot : hr metrics: application of formulas for measuring and calculating core hr issues so as to draw exact hr results and current scenario of organisation.
how do they work?
Chattybot : creating a system to coordinate the works of the members so as to make the employees to work properly and not to cause any conflict in the allocation of the work to the employees.
tell me about
Chattybot : Checking Wikipedia
No content available
None
Maths
Chattybot : Checking Wikipedia
None
tell me about Friends
Chattybot : Checking Wikipedia
Friends is an American television sitcom created by David Crane and Marta Kauffman, which aired on NBC from September 22, 1994, to May 6, 2004, lasting ten seasons. With an ensemble cast starring Jennifer Aniston, Courteney Cox, Lisa Kudrow, Matt LeBlanc, Matthew Perry and David Schwimmer, the show revolves around six friends in t